In [1]:
%cd ..

/Users/ahmedmaher/Documents/GitHub/2nd_model_calssification_API


In [2]:
from datasets import load_dataset

/Users/ahmedmaher/.pyenv/versions/3.11.12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
food_dataset=load_dataset('food101',split='train[:5000]')

In [4]:
from sklearn.model_selection import train_test_split
food_splitted=food_dataset.train_test_split(train_size=0.85)

In [5]:
labels=food_splitted['train'].features['label'].names
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [6]:
from transformers import AutoImageProcessor

In [7]:
processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224-pt22k-ft22k")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
from torchvision.transforms import RandomResizedCrop ,Compose,Normalize,Resize,ToTensor

In [9]:
size=(processor.size['height'],processor.size['width'])

In [10]:
transform=Compose([
    RandomResizedCrop(size),
    ToTensor(),
    Normalize(mean=processor.image_mean,std=processor.image_std)
])

In [11]:
def preprocess_image(samples):
    samples['pixel_values']=[transform(img.convert('RGB'))for img in samples['image']]
    del samples['image']
    return samples

In [12]:
def preprocess_image(samples):
    samples['pixel_values']=[transform(img.convert('RGB'))for img in samples['image']]
    del samples['image']
    return samples

In [13]:
food_splited_trans=food_splitted.with_transform(preprocess_image)

In [14]:
food_splited_trans["train"][0]

{'label': 81,
 'pixel_values': tensor([[[ 0.5608,  0.5922,  0.6157,  ..., -0.0431, -0.0745, -0.0902],
          [ 0.5843,  0.6000,  0.6157,  ..., -0.0510, -0.0745, -0.0902],
          [ 0.5922,  0.6078,  0.6314,  ..., -0.0588, -0.0667, -0.0745],
          ...,
          [-0.5922, -0.6314, -0.7333,  ...,  0.0980,  0.0980,  0.1137],
          [-0.5843, -0.6157, -0.7412,  ...,  0.1451,  0.1765,  0.1529],
          [-0.5765, -0.6078, -0.7255,  ...,  0.1294,  0.1686,  0.1686]],
 
         [[ 0.1765,  0.1922,  0.2078,  ..., -0.4196, -0.4353, -0.4510],
          [ 0.2078,  0.2235,  0.2392,  ..., -0.4118, -0.4353, -0.4510],
          [ 0.2392,  0.2627,  0.2863,  ..., -0.4275, -0.4275, -0.4275],
          ...,
          [-0.5922, -0.6392, -0.7412,  ..., -0.1059, -0.0980, -0.0824],
          [-0.5843, -0.6235, -0.7490,  ..., -0.0588, -0.0275, -0.0510],
          [-0.5765, -0.6157, -0.7333,  ..., -0.0824, -0.0431, -0.0431]],
 
         [[-0.1373, -0.1137, -0.0980,  ..., -0.6784, -0.7020, -0.7176]

In [15]:
from transformers import AutoModelForImageClassification, Trainer, TrainingArguments,DefaultDataCollator
data_collator=DefaultDataCollator()

In [16]:
import evaluate
import numpy as np
metric=evaluate.load("accuracy")
def compute_metrics(predictions_and_labels):
    predictions,labels=predictions_and_labels
    predictions=np.argmax(predictions,axis=1)
    return metric.compute(predictions=predictions,references=labels)

In [17]:
from transformers import logging
logging.set_verbosity_error()
model=AutoModelForImageClassification.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k',
                                                      num_labels=len(labels),
                                                      id2label=id2label,
                                                      label2id=label2id,
                                                       ignore_mismatched_sizes=True
                                                     )


In [18]:
! pip install --upgrade accelerate


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [20]:
training_args=TrainingArguments(
    output_dir='my_model',
    remove_unused_columns=False,
    learning_rate=0.01,
     per_device_train_batch_size=16,
     per_device_eval_batch_size=16,
     num_train_epochs=20,
      weight_decay=0.01,
      save_strategy="epoch",
      logging_steps=10,
      eval_strategy="epoch",
     push_to_hub=False,
    metric_for_best_model="accuracy"

)

In [21]:
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=food_splited_trans["train"],
    eval_dataset=food_splited_trans["test"]
)

In [22]:
trainer.train()

/Users/ahmedmaher/.pyenv/versions/3.11.12/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 5.2326, 'grad_norm': 13.13155746459961, 'learning_rate': 0.009983082706766917, 'epoch': 0.03759398496240601}
{'loss': 2.4501, 'grad_norm': 5.916259288787842, 'learning_rate': 0.009964285714285715, 'epoch': 0.07518796992481203}
{'loss': 2.1728, 'grad_norm': 3.915470600128174, 'learning_rate': 0.009945488721804513, 'epoch': 0.11278195488721804}
{'loss': 2.0055, 'grad_norm': 1.7062156200408936, 'learning_rate': 0.009926691729323308, 'epoch': 0.15037593984962405}
{'loss': 2.0589, 'grad_norm': 3.2155518531799316, 'learning_rate': 0.009907894736842105, 'epoch': 0.18796992481203006}
{'loss': 2.0596, 'grad_norm': 1.9387056827545166, 'learning_rate': 0.009889097744360902, 'epoch': 0.22556390977443608}
{'loss': 2.0074, 'grad_norm': 2.6542999744415283, 'learning_rate': 0.0098703007518797, 'epoch': 0.2631578947368421}
{'loss': 1.976, 'grad_norm': 2.1936681270599365, 'learning_rate': 0.009851503759398497, 'epoch': 0.3007518796992481}
{'loss': 1.9197, 'grad_norm': 1.4799751043319702, 'learn

/Users/ahmedmaher/.pyenv/versions/3.11.12/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 1.8384, 'grad_norm': 0.8360782265663147, 'learning_rate': 0.009494360902255639, 'epoch': 1.0150375939849625}
{'loss': 1.872, 'grad_norm': 1.3109667301177979, 'learning_rate': 0.009475563909774436, 'epoch': 1.0526315789473684}
{'loss': 1.9375, 'grad_norm': 0.6895990371704102, 'learning_rate': 0.009456766917293233, 'epoch': 1.0902255639097744}
{'loss': 1.8768, 'grad_norm': 0.8945196270942688, 'learning_rate': 0.00943796992481203, 'epoch': 1.1278195488721805}
{'loss': 1.8431, 'grad_norm': 1.072113275527954, 'learning_rate': 0.009419172932330827, 'epoch': 1.1654135338345863}
{'loss': 1.9231, 'grad_norm': 1.0721923112869263, 'learning_rate': 0.009400375939849624, 'epoch': 1.2030075187969924}
{'loss': 1.9263, 'grad_norm': 0.9410565495491028, 'learning_rate': 0.009381578947368421, 'epoch': 1.2406015037593985}
{'loss': 1.8976, 'grad_norm': 0.6659327149391174, 'learning_rate': 0.009362781954887218, 'epoch': 1.2781954887218046}
{'loss': 1.7912, 'grad_norm': 1.3833417892456055, 'learning

/Users/ahmedmaher/.pyenv/versions/3.11.12/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

/Users/ahmedmaher/.pyenv/versions/3.11.12/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 1.4072, 'grad_norm': 0.9435099363327026, 'learning_rate': 0.0009981203007518797, 'epoch': 18.007518796992482}
{'loss': 1.5321, 'grad_norm': 0.7994762659072876, 'learning_rate': 0.0009793233082706766, 'epoch': 18.045112781954888}
{'loss': 1.5314, 'grad_norm': 0.6893203854560852, 'learning_rate': 0.0009605263157894736, 'epoch': 18.082706766917294}
{'loss': 1.3416, 'grad_norm': 1.2285258769989014, 'learning_rate': 0.0009417293233082707, 'epoch': 18.1203007518797}
{'loss': 1.5722, 'grad_norm': 0.84882652759552, 'learning_rate': 0.0009229323308270676, 'epoch': 18.157894736842106}
{'loss': 1.5146, 'grad_norm': 1.139862060546875, 'learning_rate': 0.0009041353383458646, 'epoch': 18.195488721804512}
{'loss': 1.4878, 'grad_norm': 0.9175910949707031, 'learning_rate': 0.0008853383458646616, 'epoch': 18.23308270676692}
{'loss': 1.3958, 'grad_norm': 1.3214141130447388, 'learning_rate': 0.0008665413533834586, 'epoch': 18.270676691729324}
{'loss': 1.5853, 'grad_norm': 1.2581876516342163, 'lea

/Users/ahmedmaher/.pyenv/versions/3.11.12/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 1.5109, 'grad_norm': 3.674574851989746, 'learning_rate': 0.0004906015037593986, 'epoch': 19.022556390977442}
{'loss': 1.4587, 'grad_norm': 0.7544513940811157, 'learning_rate': 0.00047180451127819555, 'epoch': 19.06015037593985}
{'loss': 1.4073, 'grad_norm': 1.006731629371643, 'learning_rate': 0.0004530075187969925, 'epoch': 19.097744360902254}
{'loss': 1.3944, 'grad_norm': 1.30697762966156, 'learning_rate': 0.00043421052631578954, 'epoch': 19.13533834586466}
{'loss': 1.5146, 'grad_norm': 1.0449272394180298, 'learning_rate': 0.0004154135338345865, 'epoch': 19.172932330827066}
{'loss': 1.4788, 'grad_norm': 0.9277982711791992, 'learning_rate': 0.0003966165413533835, 'epoch': 19.210526315789473}
{'loss': 1.4037, 'grad_norm': 2.6692001819610596, 'learning_rate': 0.0003778195488721805, 'epoch': 19.24812030075188}
{'loss': 1.4085, 'grad_norm': 3.5717716217041016, 'learning_rate': 0.0003590225563909774, 'epoch': 19.285714285714285}
{'loss': 1.4717, 'grad_norm': 1.0418310165405273, 'le

TrainOutput(global_step=5320, training_loss=1.631724436838824, metrics={'train_runtime': 9867.4876, 'train_samples_per_second': 8.614, 'train_steps_per_second': 0.539, 'train_loss': 1.631724436838824, 'epoch': 20.0})